## Setup

In [29]:
import os, glob, subprocess, requests
import pickle
from osgeo import gdal
import rasterio as rio
from rasterio.merge import merge
from rasterio.plot import show
# import georasters as gr
import numpy as np
import pandas as pd
from math import floor, ceil
import matplotlib.pyplot as plt
from matplotlib.collections import PatchCollection
import seaborn as sns
from shapely.geometry import Point
import neonutilities as nu
import geopandas as gpd 
import logging
from typing import List

from neonutilities.aop_download import validate_dpid,validate_site_format,validate_neon_site
from neonutilities.helper_mods.api_helpers import get_api
from neonutilities import __resources__
from neonutilities.helper_mods.api_helpers import get_api, download_file
from neonutilities.helper_mods.metadata_helpers import convert_byte_size
from neonutilities.get_issue_log import get_issue_log
from neonutilities.citation import get_citation

from neonutilities.aop_download import *

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

### Loop and download Vegetation Structure Data (DP1.10098.001)

In [ ]:
site_names = ['DELA','LENO','TALL','BONA','DEJU','HEAL','SRER','SJER','SOAP',
              'TEAK','CPER','NIWO','RMNP','DSNY','OSBS','JERC','PUUM','KONZ',
              'UKFS','SERC','HARV','UNDE','BART','JORN','DCFS','NOGP','WOOD',
              'GUAN','LAJA','GRSM','ORNL','CLBJ','MOAB','ONAQ','BLAN','MLBS',
              'SCBI','ABBY','WREF','STEI','TREE','YELL']

site_worked = []
site_failed = []

for site_name in site_names:

# site_name = site_names[0]
    try:
        veg_dict = nu.load_by_product(dpid="DP1.10098.001", 
                                    site=site_name, 
                                    package="basic", 
                                    release="RELEASE-2025",
                                    check_size=False)

        # Save dictionary
        with open('/data/chloris/NEON/DP1.10098/' + site_name + '.pkl', 'wb') as f:
            pickle.dump(veg_dict, f)
        site_worked.append(site_name)

    except Exception as e:
        print(f"Failed for site {site_name}: {e}")
        site_failed.append(site_name)

In [80]:
flist = os.listdir('/data/chloris/NEON/VST/')
flist = [fi for fi in flist if fi.endswith('_single_bole_trees.parquet')]

total = 0
df_list = []
for fname in flist:

	fname = os.path.join('/data/chloris/NEON/VST/', fname)

	df = pd.read_parquet(fname)

	if df.shape[0] > 0:
		df_list.append(df)

	total = total + df.shape[0]

In [81]:
df_out = pd.concat(df_list, ignore_index=True)

In [84]:
len(np.unique(df_out.individualID))

In [87]:
plt.hist(df_out.stemDiameter,bins = 50)

In [71]:
fname = os.path.join('/data/chloris/NEON/VST/', flist[0])

gpd.read_parquet(fname).to_file('/data/chloris/NEON/DELA.fgb')

### Functions to wrangle veg structure data into a gdf w/ geolocated trees

In [ ]:
def make_veg_gdf(veg_dict: dict) -> gpd.GeoDataFrame:
	"""
	Takes the vegetation structure dict, finds the unique reference points,
	pulls their spatial reference information using the NEON "locations" API,
	and computes the location of individual stems.

	Notes:
	1. Stems without a pointID are discarded.
	2. If the points don't all belong to the same utm zone, the function will return a failure.  

	Parameters:
	veg_dict (pd.DataFrame): DataFrame containing 'easting' and 'northing' columns.

	Returns:
	The original dict with the spatial ref information added. 
	"""

	# Pull out the "mapping and tagging" dataframe
	veg_map_all = veg_dict["vst_mappingandtagging"]

	# Filter out points that have no pointID and reindex.
	veg_map = veg_map_all.loc[veg_map_all["pointID"] != ""]
	veg_map = veg_map.reindex()

	# Create a unique identifier for each point
	veg_map["points"] = veg_map["namedLocation"] + "." + veg_map["pointID"]

	# Make a list of the unique points 
	veg_points = list(set(list(veg_map["points"])))

	# Loop every point, pull the spatial ref info, and store in lists
	easting = []
	northing = []
	coord_uncertainty = []
	elev_uncertainty = []
	utm_zone = []
	for i in veg_points:
		vres = requests.get("https://data.neonscience.org/api/v0/locations/"+i)
		vres_json = vres.json()
		easting.append(vres_json["data"]["locationUtmEasting"])
		northing.append(vres_json["data"]["locationUtmNorthing"])
		props = pd.DataFrame.from_dict(vres_json["data"]["locationProperties"])
		cu = props.loc[props["locationPropertyName"]=="Value for Coordinate uncertainty"]["locationPropertyValue"]
		coord_uncertainty.append(cu[cu.index[0]])
		eu = props.loc[props["locationPropertyName"]=="Value for Elevation uncertainty"]["locationPropertyValue"]
		elev_uncertainty.append(eu[eu.index[0]])
		utm_zone.append(32600+int(vres_json["data"]["locationUtmZone"]))

	# Create a dataframe with the spatial info of the reference points
	pt_dict = dict(points=veg_points, 
	easting=easting,
	northing=northing,
	coordinateUncertainty=coord_uncertainty,
	elevationUncertainty=elev_uncertainty,
	utm_zone = utm_zone)
	pt_df = pd.DataFrame.from_dict(pt_dict)
	pt_df.set_index("points", inplace=True)

	# Add the reference info to the veg map
	veg_map = veg_map.join(pt_df, 
	on="points", 
	how="inner")

	# Compute the Easting of each stem 
	veg_map["stemEasting"] = (veg_map["easting"]
	+ veg_map["stemDistance"]
	* np.sin(veg_map["stemAzimuth"]
	* np.pi / 180))

	# Compute the Northing of each stem
	veg_map["stemNorthing"] = (veg_map["northing"]
	+ veg_map["stemDistance"]
	* np.cos(veg_map["stemAzimuth"]
	* np.pi / 180))

	# Compute the stem uncertainties
	veg_map["stemCoordinateUncertainty"] = veg_map["coordinateUncertainty"] + 0.6
	veg_map["stemElevationUncertainty"] = veg_map["elevationUncertainty"] + 1.5

	# Test that all points are in the same UTM zone
	if len(set(veg_map["utm_zone"])) != 1:
		raise ValueError("Points in the veg map are in different UTM zones! Rectify before proceeding.")	
	else:
		# Define the crs
		crs = f"EPSG:{veg_map['utm_zone'].values[0]}"
		print(f"veg_map converted to gdf with crs {crs}")

		# Create list of shapely points for the gdf 	
		geometry = [Point(xy) for xy in zip(veg_map["stemEasting"], veg_map["stemNorthing"])]

		# Convert veg_map to a gdf 
		veg_map = gpd.GeoDataFrame(veg_map, crs=crs, geometry=geometry)

		veg_dict["vst_apparentindividual"].set_index("individualID", inplace=True)
		veg_gdf = veg_map.join(veg_dict["vst_apparentindividual"],
		on="individualID",
		how="inner",
		lsuffix="_MAT",
		rsuffix="_AI")

		return veg_gdf


def filter_veg_gdf(veg_gdf: gpd.GeoDataFrame,
				require_dbh:bool = True,
				single_bole:bool = True,	
				only_most_recent:bool = True) -> gpd.GeoDataFrame:
	"""
	Filters down a vegetation gdf to just the trees, with options to 
	1. drop trees without a dbh measurement,
	2. keep only single bole trees,
	3. keep only the most recent measurement of each tree.

	Returns the filtered gdf.
	"""

    # Filter to only trees that have a dbh value
	if require_dbh:
		veg_gdf = veg_gdf.loc[~veg_gdf["stemDiameter"].isna()]	

	# Drop duplicated measurements 
	dupe_test_cols = ['date_AI','individualID','scientificName','taxonID','family',
					'growthForm','plotID_AI','pointID','stemDiameter',
					'maxBaseCrownDiameter','stemEasting','stemNorthing']
	veg_gdf = veg_gdf.drop_duplicates(subset = dupe_test_cols)

	# Cut down to just the tree growth forms 
	tree_gdf = veg_gdf[veg_gdf['growthForm'].str.contains('tree|sapling', regex=True)]

	# Cut down to just single bole trees
	if single_bole:
		tree_gdf = tree_gdf[(tree_gdf['growthForm']=='single bole tree')]

	# Convert 'date_AI' to datetime if it's not already
	tree_gdf.loc[:, 'date_AI'] = pd.to_datetime(tree_gdf['date_AI'])

	# Sort the DataFrame by 'individualID' and 'date_AI' in descending order
	tree_gdf = tree_gdf.sort_values(by=['individualID', 'date_AI'], ascending=[True, False])

	# Keep only the most recent entry for each 'individualID'
	if only_most_recent:
		tree_gdf = tree_gdf.drop_duplicates(subset='individualID', keep='first')
		
	return tree_gdf
 

def nu_list_available_dates(dpid:str, site:str) -> pd.DataFrame:

    """
		NOTE: This is an internal hack of the original function to return a dataframe instead of printing
        
        nu_list_available_dates displays the available releases and dates for a given product and site
        --------
         Inputs:
             dpid: the data product code (eg. 'DP3.30015.001' - CHM)
             site: the 4-digit NEON site code (eg. 'JORN')
        --------
        Returns:
        prints the Release Tag (or PROVISIONAL) and the corresponding available dates (YYYY-MM) for each tag
    --------
        Usage:
        --------
        >>> list_available_dates('DP3.30015.001','JORN')
        RELEASE-2025 Available Dates: 2017-08, 2018-08, 2019-08, 2021-08, 2022-09

        >>> list_available_dates('DP3.30015.001','HOPB')
        PROVISIONAL Available Dates: 2024-09
        RELEASE-2025 Available Dates: 2016-08, 2017-08, 2019-08, 2022-08

        >>> list_available_dates('DP1.10098.001','HOPB')
        ValueError: There are no data available for the data product DP1.10098.001 at the site HOPB.
    """
    product_url = "https://data.neonscience.org/api/v0/products/" + dpid
    response = get_api(api_url=product_url)  # add input for token?

    # raise value error and print message if dpid isn't formatted as expected
    validate_dpid(dpid)

    # raise value error and print message if site is not a 4-letter character
    site = site.upper()  # make site upper case (if it's not already)
    validate_site_format(site)

    # raise value error and print message if site is not a valid NEON site
    validate_neon_site(site)

    # check if product is active
    if response.json()["data"]["productStatus"] != "ACTIVE":
        raise ValueError(
            f"NEON {dpid} is not an active data product. See https://data.neonscience.org/data-products/{dpid} for more details."
        )

    # get available releases & months:
    for i in range(len(response.json()["data"]["siteCodes"])):
        if site in response.json()["data"]["siteCodes"][i]["siteCode"]:
            available_releases = response.json()["data"]["siteCodes"][i][
                "availableReleases"
            ]

    # display available release tags (including provisional) and dates for each tag
    try:
        availables_list = []
        for entry in available_releases:
            release = entry["release"]
            available_months_str = ", ".join(entry["availableMonths"])
            available_months = [x.strip() for x in available_months_str.split(',')]
            for available_month in available_months:
                availables_list.append({'status':release,'date':available_month})
        available_df = pd.DataFrame(availables_list)
        return(available_df)
    except UnboundLocalError:
        # if the available_releases variable doesn't exist, this error will show up:
        # UnboundLocalError: local variable 'available_releases' referenced before assignment
        raise ValueError(
            f"There are no NEON data available for the data product {dpid} at the site {site}."
        )
    
def nu_aop_file_names(
	dpid:str,
	site:str,
	year:int,
	token="",
	include_provisional=True,
	check_size=False,
	savepath=None) -> List:
	"""returns names of files in the specified AOP data product, site, and year. 
	   changes the standard NEON paths to the local paths if savepath specified. """

	# raise value error and print message if dpid isn't formatted as expected
	validate_dpid(dpid)

	# raise value error and print message if dpid isn't formatted as expected
	validate_aop_dpid(dpid)

	# raise value error and print message if field spectra data are attempted
	check_field_spectra_dpid(dpid)

	# raise value error and print message if site is not a 4-letter character
	site = site.upper()  # make site upper case (if it's not already)
	validate_site_format(site)

	# raise value error and print message if site is not a valid NEON site
	validate_neon_site(site)

	# raise value error and print message if year input is not valid
	year = str(year)  # cast year to string (if it's not already)
	validate_year(year)

	# if token is an empty string, set to None
	if token == "":
		token = None

	# query the products endpoint for the product requested
	response = get_api("https://data.neonscience.org/api/v0/products/" + dpid, token)

	# exit function if response is None (eg. if no internet connection)
	if response is None:
		logging.info("No response from NEON API. Check internet connection")

	# check that token was used
	if token and "x-ratelimit-limit" in response.headers:
		check_token(response)
		# if response.headers['x-ratelimit-limit'] == '200':
		#     print('API token was not recognized. Public rate limit applied.\n')

	# get the request response dictionary
	response_dict = response.json()

	# error message if dpid is not an AOP data product
	check_aop_dpid(response_dict, dpid)

	# replace collocated site with the AOP site name it's published under
	site = get_shared_flights(site)

	# get the urls for months with data available, and subset to site & year
	site_year_urls = get_site_year_urls(response_dict, site, year)

	# error message if nothing is available
	if len(site_year_urls) == 0:
		logging.info(
			f"There are no NEON {dpid} data available at the site {site} in {year}.\nTo display available dates for a given data product and site, use the function list_available_dates()."
		)
		# print("There are no data available at the selected site and year.")

	# get file url dataframe for the available month urls
	file_url_df, releases = get_file_urls(site_year_urls, token=token)

	# get the number of files in the dataframe, if there are no files to download, return
	if len(file_url_df) == 0:
		# print("No data files found.")
		logging.info("No NEON data files found.")
		# return

	# NOTE: provisional filtering has been silenced for now. 
	# if 'PROVISIONAL' in releases and not include_provisional:
	# if include_provisional:
	# 	# log provisional included message
	# 	# logging.info(
	# 	# 	"Provisional NEON data are included. To exclude provisional data, use input parameter include_provisional=False."
	# 	# )
	# else:
	# 	# log provisional not included message and filter to the released data
	# 	# logging.info(
	# 	#     "Provisional data are not included. To download provisional data, use input parameter include_provisional=True.")
	# 	file_url_df = file_url_df[file_url_df["release"] != "PROVISIONAL"]
	# 	if len(file_url_df) == 0:
	# 		logging.info(
	# 			"NEON Provisional data are not included. To download provisional data, use input parameter include_provisional=True."
	# 		)

	num_files = len(file_url_df)
	if num_files == 0:
		logging.info(
			"No NEON data files found. Available data may all be provisional. To download provisional data, use input parameter include_provisional=True."
		)
		# return

	# get the total size of all the files found
	download_size_bytes = file_url_df["size"].sum()
	# print(f'download size, bytes: {download_size_bytes}')
	download_size = convert_byte_size(download_size_bytes)
	# print(f'download size: {download_size}')

	# report data download size and ask user if they want to proceed
	if check_size:
		if (
			input(
				f"Continuing will download {num_files} NEON data files totaling approximately {download_size}. Do you want to proceed? (y/n) "
			)
			.strip()
			.lower()
			!= "y"
		):  # lower or upper case 'y' will work
			print("Download halted.")
			# return

	# Make the list of files as they should appear on the local file system and return 
	files = list(file_url_df["url"])
	if savepath is not None:
		files = [f"{savepath.rstrip('/')}/{fi.lstrip('https://storage.googleapis.com')}" for fi in files]
	return files 

In [56]:
dpid = "DP3.30015.001"
site = "DELA"
year = 2017
dp3_path = '/data/chloris/NEON/DP3.30015.001/'

out_path = '/data/chloris/NEON/CHM/'

site_names = ['DELA','LENO','TALL','BONA','DEJU','HEAL','SRER','SJER','SOAP',
              'TEAK','CPER','NIWO','RMNP','DSNY','OSBS','JERC','PUUM','KONZ',
              'UKFS','SERC','HARV','UNDE','BART','JORN','DCFS',]
 
site_names = ['NOGP','WOOD','GUAN','LAJA','GRSM','ORNL','CLBJ','MOAB','ONAQ','BLAN','MLBS','SCBI','ABBY','WREF','STEI','TREE','YELL']

for site in site_names:

	# Get available dates 
	available_dates_df = nu_list_available_dates(dpid=dpid, site=site)

	# Parse out the years
	available_dates_df['year'] = available_dates_df['date'].str.split('-').str[0].astype(int)

	# Loop the years
	for year in available_dates_df['year'].unique():

		# Figure out the output name
		out_fname = f"{out_path.rstrip('/')}/{site}_{year}_CHM.tif"

		print(f'Merging {site} {year}')

		# Get the list of files for this site and year 
		flist = nu_aop_file_names(
			dpid = dpid,
			site = site,
			year = year,
			savepath=dp3_path)

		# Grab the tifs only
		tif_list = [fi for fi in flist if fi.endswith('.tif')]

		# Write the file names to a text file
		tif_list_file = f"{out_path.rstrip('/')}/file_lists/{site}_{year}_tif_list.txt"
		with open(tif_list_file, "w") as f:
			for tif in tif_list:
				f.write(f"{tif}\n")

		#Construct the gdal string 
		gdal_cmd = (
			f"gdalwarp --optfile {tif_list_file} -co TILED=YES "
			f"-co BLOCKXSIZE=1024 -co BLOCKYSIZE=1024 {out_fname}"
			f" -co COMPRESS=LZW -co PREDICTOR=2"
		)

		subprocess.run(gdal_cmd, shell=True)


## Canopy Height Model Data (CHM - DP3.30015.001)

In [ ]:
from datetime import datetime
import importlib_resources
import logging
import pandas as pd
import numpy as np
import os
import re
from time import sleep
from tqdm import tqdm
from typing import List
# local imports
from neonutilities import __resources__
from neonutilities.helper_mods.api_helpers import get_api, download_file
from neonutilities.helper_mods.metadata_helpers import convert_byte_size
from neonutilities.get_issue_log import get_issue_log
from neonutilities.citation import get_citation

from neonutilities.aop_download import *


In [23]:
files

In [16]:
file_url_df

The most recent data is from 2024, so we can use that. This is still available provisionally  (eg. not as part of a NEON data release), because everything generated in 2024 is provisional in 2025 (there is a year lag period). The data should be good as long as there are no reported issues in the Issue Logs (see the Data Portal). Next we can see the extent of the data. You don't need to run this step, but it can be helpful for context.

sjer_exts = nu.get_aop_tile_extents('DP3.30015.001','SJER','2024')

Use the Eastings and Northings determined from the vegetation data to download only the tiles that overlay with the veg data points. Use `by_tile_aop` to do this:

In [ ]:
sjer_exts = nu.get_aop_tile_extents('DP3.30015.001','SJER','2024')

In [ ]:
sjer_exts

In [ ]:
nu.by_tile_aop(dpid="DP3.30015.001", site="SJER", year="2024", 
          easting=list(trees_sb_latest.adjEasting), 
          northing=list(trees_sb_latest.adjNorthing),
          include_provisional=True,
          savepath=os.getcwd())

Let's see what files were downloaded:

In [ ]:
chm_files = []
for root, dirs, files in os.walk("DP3.30015.001"):
    for file in files:
        if file.endswith(".tif"):
            print(file)
            chm_files.append(os.path.join(root,file))

To simplify, you can pare down the data again to only look at the central rectangular region. Those are the following 6 tiles:

We can save this merged CHM raster to a new geotiff file, called `SJER_2024_CHM_merged.tif`:

In [ ]:
# Update metadata for the merged CHM 
output_meta = chm.meta.copy()
output_meta.update({
    "driver": "GTiff",
    "height": chm_mosaic.shape[1],
    "width": chm_mosaic.shape[2],
    "transform": chm_transform,
})

# Write the merged raster to a new GeoTIFF file
output_path = 'SJER_2024_CHM_merged.tif'
with rio.open(output_path, "w", **output_meta) as dest:
    dest.write(chm_mosaic)

Now filter the TOS dataset to only include the values inside this smaller extent.

You may decide to keep only the Live trees, depending on your appication. As you can see, there are other categories, including Standing dead; Live, physically damaged; and Live, disease damaged.

Plot all the trees - can show the taxonID and growthForms, or other information you are interested in:

In [ ]:
# Use the 'hue' argument to color by species
sns.lmplot(x="adjEasting", y="adjNorthing", data=veg, fit_reg=False, hue='taxonID', legend=False);
# anchor the legend to outside right of the plot
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);
plt.xticks(rotation=90);

# Use the 'hue' argument to color by species
sns.lmplot(x="adjEasting", y="adjNorthing", data=veg, fit_reg=False, hue='growthForm', legend=False);
# anchor the legend to outside right of the plot
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);
plt.xticks(rotation=90);

Below are links to each of the USDA plant profiles. 

USDA Plant Profiles:
- https://plants.usda.gov/core/profile?symbol=qudo - Quercus douglasii (blue oak)
- https://plants.usda.gov/core/profile?symbol=cecu - Ceanothus cuneatus (buckbrush)
- https://plants.usda.gov/core/profile?symbol=QUWI2 - Quercus wislizeni (interior live oak)
- https://plants.usda.gov/core/profile?symbol=PISA2 - Pinus sabiniana (california foothill pine)
- https://plants.usda.gov/core/profile?symbol=CELE2 - Ceanothus leucodermis (chaparral whitethorn) *shrub
- https://plants.usda.gov/core/profile?symbol=LUAL4 - Lupinus albifrons (silver lupine) *shrub
- https://plants.usda.gov/core/profile?symbol=ARVIM - Arctostaphylos viscida (Mariposa manzanita) *shrub tree
- https://plants.usda.gov/core/profile?symbol=RHIL - Rhamnus ilicifolia (hollyleaf redberry) *shrub tree

Plot only the single bole trees (`trees_sb_subset`):

In [ ]:
sns.lmplot(x="adjEasting", y="adjNorthing", data=trees_sb_subset, fit_reg=False, hue='taxonID', legend=False);
# anchor the legend to outside right of the plot
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.);
plt.title('Single Bole Trees'); plt.xticks(rotation=90);

Most of the single-bole trees are Oaks (QUDO), (QUWI2). 

Next, extract the CHM values corresponding to each of the veg data points. You can do this in several different ways and with several different Python packages. We will show using `rasterio` (imported as `rio`) as well as `georasters`, which provides more advanced extraction options.

In [ ]:
# read in with rasterio
chm_merged = rio.open("SJER_2024_CHM_merged.tif")

Extract the CHM value corresponding to each of the trees in the TOS single-bole tree dataset (trees_sb_subset):

In [ ]:
valCHM = list(rio.sample.sample_gen(chm_merged, 
                                tuple(zip(trees_sb_subset["adjEasting"], 
                                          trees_sb_subset["adjNorthing"])),
                                masked=True))

fig, ax = plt.subplots()

ax.plot((0,50), (0,50), linewidth=1, color="black")
ax.scatter(trees_sb_subset.height, valCHM, s=1.5)

ax.set_xlabel("TOS Height")
ax.set_ylabel("CHM Height")

plt.show()

There is fairly good alignment between the CHM heights and TOS heights, with the exception of a few outliers. Keep in mind that anything above the 1-1 line (CHM height > TOS height) might represent understory, as the CHM represents the canopy height. There may be other reasons for outliers - including geographic mismatch. Say the point locations between the TOS and Lidar-derived positions are off by a meter, the lidar could be detecting ground where the TOS data includes a tree. In the plot above we can see several examples where the CHM height is 0 and the TOS height is > 5 m. You may need to either adjust point locations, or filter out these outliers to exclude them from the model. 

You can also look at the stem diameter v. CHM height. There should be a positive correlation, but more scatter is expected.

In [ ]:
# make a plot of stemDiameter v. CHM height
fig, ax = plt.subplots()

ax.scatter(trees_sb_subset.stemDiameter, valCHM, s=1.5)

ax.set_xlabel("TOS Stem Diameter (cm)")
ax.set_ylabel("CHM Height (m)")

plt.show()

Depending on your approach, you may need to filter out outliers and/or remove the understory plants. Remember the CHM represents the canopy trees, so can mask the understory. See the Veg Structure and CHM lesson (https://www.neonscience.org/resources/learning-hub/tutorials/tree-heights-veg-structure-chm) for some ideas on further filtering.

We can also make a function for plotting the rasters, this is optional but another way to display the CHM data.

In [ ]:
def plot_raster(band_array,image_extent,title,cmap_title,colormap,colormap_limits):
    plt.imshow(band_array,extent=image_extent)
    cbar = plt.colorbar(); plt.set_cmap(colormap); plt.clim(colormap_limits)
    cbar.set_label(cmap_title,rotation=270,labelpad=20)
    plt.title(title); ax = plt.gca()
    ax.ticklabel_format(useOffset=False, style='plain') 
    rotatexlabels = plt.setp(ax.get_xticklabels(),rotation=90)

In [ ]:
pip install georasters

In [ ]:
import georasters as gr

Use the Python `georasters` package (imported as `gr`) to read in the mosaicked raster. This gives us a little more flexibility for extraction (eg. we can select the max CHM value within a given radius).

In [ ]:
sjer_chm = gr.from_file('SJER_2024_CHM_merged.tif')
sjer_chm_array=sjer_chm.raster.data
sjer_chm_extent=[sjer_chm.bounds[0],sjer_chm.bounds[2],sjer_chm.bounds[1],sjer_chm.bounds[3]]

In [ ]:
plot_raster(sjer_chm_array,sjer_chm_extent,'SJER CHM Subset','Canopy Height (m)','Greens',[0,30])

Let's select the CHM height, and the maximum CHM height within a 3 m radius. This may help with cases where there is geographic mismatch between the TOS and CHM data.

In [ ]:
tree_training_df = trees_sb_subset[(np.isfinite(trees_sb_subset['adjEasting'])) & (np.isfinite(trees_sb_subset['adjNorthing']))].reset_index(drop=True)
for i,row in tree_training_df.iterrows():
    chm_height = sjer_chm.map_pixel(tree_training_df.loc[i,'adjEasting'],tree_training_df.loc[i,'adjNorthing'])  
    chm_height_max3 = sjer_chm.extract(tree_training_df.loc[i,'adjEasting'],tree_training_df.loc[i,'adjNorthing'],radius=3).max()
    tree_training_df.at[i,'chm_height'] = chm_height
    tree_training_df.at[i,'chm_height_max3'] = chm_height_max3
tree_training_df[['date_AI','individualID','scientificName','taxonID','family','growthForm','plantStatus','stemDiameter','height','chm_height','chm_height_max3']].head(5)

As you can see with the 3rd record (`NEON.PLA.D17.SJER.00037`), the `chm_height` is 0, but the `chm_height_max3` is 6.18, much closer to the TOS measured height of 6.5. This may be a better variable to use.

In [ ]:
# Plot TOS v. CHM heights for comparison, colored by taxonID:
sns.lmplot(tree_training_df, x='height', y='chm_height', hue='taxonID', fit_reg=False); ax = plt.gca()
#plot 1-1 line for comparison
compare_line = np.arange(0,ceil(np.nanmax([tree_training_df['height'],tree_training_df['chm_height_max3']])))
ax.plot(compare_line,compare_line,'--',color='red')
ax.axis('equal')
ax.set_title('AOP v. TOS Tree Heights, CHM Pixel Height')
ax.set_xlabel('TOS Height (m)')
ax.set_ylabel('AOP CHM Height (m)');

There is one Oak with an anomalously tall height of 57 m. We can look at this more closely to see what's going on.

In [ ]:
tree_training_df[tree_training_df['height']>50][['height','chm_height','chm_height_max3','stemDiameter','dataQF_AI']]

Legacy data was collected before data collection app (Fulcrum) was available, so are subject to more human/recorder error. This may have been off by a decimal place. While the stem diameter seems reasonable, you may choose to remove this record, as we do not expect any Oak trees this tall.

In [ ]:
tree_training_df = tree_training_df[tree_training_df['height']<50]

The CHM heights tend to be lower than field-measured heights, which fits with this plot. We interpret values above the 1-1 line to be understory trees, which we can filter out according to some threshold. Let's try a ratio of 0.75 to start:

In [ ]:
training_df_filtered = tree_training_df.loc[(tree_training_df['height'] > 0.75*tree_training_df['chm_height']) & (tree_training_df['chm_height']>2)]
sns.lmplot(training_df_filtered, x='height', y='chm_height_max3', hue='taxonID', fit_reg=False);
plt.axis('equal');

You may also wish to remove the PISA2 outlier, where the TOS height is ~ 15m but the chm_height_max3 < 5m. This may be due to geographic mismatch, or some other manual error.

In [ ]:
training_df_filtered[(training_df_filtered['chm_height_max3']<5) & (training_df_filtered['taxonID'] == 'PISA2')][['individualID','taxonID','height','chm_height','chm_height_max3','stemDiameter']]

This may be off due to a geographic mismatch, or some other error. Remove this record as well, to be safe.

In [ ]:
training_df_filtered = training_df_filtered[training_df_filtered['individualID'] != 'NEON.PLA.D17.SJER.04443']

In [ ]:
len(training_df_filtered)

### Biomass calculations - allometric equations

Now that we've identified and filtered out the outliers, let's calculate biomass for the filtered dataframe. For multi-bole trees, calculate biomass for each bole according to the standard formula, and then aggregate by `individualID`. 

**USDA Plant Profiles:**
- https://plants.usda.gov/core/profile?symbol=qudo - Quercus douglasii (blue oak)
- https://plants.usda.gov/core/profile?symbol=QUWI2 - Quercus wislizeni (interior live oak)
- https://plants.usda.gov/core/profile?symbol=PISA2 - Pinus sabiniana (california foothill pine)

**Biomass equation:**

$Biomass = exp(\beta_0 + \beta_1 ln(DBH))$

where
- Biomass = total aboveground biomass (kg) for trees 2.5cm dbh and larger
- DBH = diameter at breast height (cm)

Make a dataframe containing the allometry parameters. We suggest using the following references to obtain these values:

Updated generalized biomass equations for North American tree species.
David C. Chojnacky, Linda S. Heath, Jennifer C. Jenkins 2013
https://academic.oup.com/forestry/article-abstract/87/1/129/602137?redirectedFrom=fulltext

Comprehensive database of diameter-based biomass regressions for North American tree species. 
Jennifer C. Jenkins, David C. Chojnacky, Linda S. Heath, Richard A. Birdsey 2004
https://research.fs.usda.gov/treesearch/7058

In [ ]:
allometry_params_df = pd.DataFrame(columns=['speciesGroup','taxonID','B0','B1'])
allometry_params_df.loc[len(allometry_params_df)] = ['Hard maple/oak/hickory/beech','QUDO',-2.0127,2.4342]
allometry_params_df.loc[len(allometry_params_df)] = ['Hard maple/oak/hickory/beech','QUWI2',-2.0127,2.4342]
allometry_params_df.loc[len(allometry_params_df)] = ['Pine','PISA2',-2.5356,2.4349]
allometry_params_df

Now that we've defined the allometry parameters, we can write a function to calculate biomass and apply this to the training dataframe:

In [ ]:
def calc_biomass(B0,B1,DBH):
    biomass = np.exp(B0+B1*np.log(DBH))
    return biomass

for i,row in tree_training_df.iterrows():
    #find allometry params corresponding to taxonID
    allometry_params = allometry_params_df.loc[(allometry_params_df['taxonID'] == tree_training_df.loc[i,'taxonID'])]
    B0 = allometry_params['B0'].item(); #print(B0)
    B1 = allometry_params['B1'].item(); #print(B1)
    DBH = tree_training_df.loc[i,'stemDiameter']
    tree_training_df.at[i,'biomass'] = calc_biomass(B0,B1,DBH)

Finally, save the training dataframe to a csv. You can select fewer or more columns as desired. You can expand upon this by adding region properties from the CHM data. This will be expanded upon in a later iteration.

In [ ]:
final_training_df = tree_training_df[['date_AI','individualID','adjEasting','adjNorthing','taxonID','growthForm','plantStatus','stemDiameter','height','chm_height_max3','chm_height','biomass']]
final_training_df.head()

In [ ]:
final_training_df.to_csv('tree_training_df.csv',index=False)

Now you can use this df as a trianing dataset. You may choose to further filter by only including Live trees, or 